## Web scrapping using python

#### References
1. [Practical Introduction to Web Scraping in Python](https://realpython.com/python-web-scraping-practical-introduction/)
2. [Web Scraping using Python](https://www.datacamp.com/community/tutorials/web-scraping-using-python)

In [1]:
# $ python3 -m venv venv
# $ . ./venv/bin/activate

In [2]:
#!pip install --upgrade pip

In [3]:
#Better
#!pip install requests BeautifulSoup4 fire


     |████████████████████████████████| 81kB 2.1MB/s eta 0:00:011
  Created wheel for fire: filename=fire-0.3.1-py2.py3-none-any.whl size=111005 sha256=a27afeb10c33782c86c2f45fd8481a7c079c21b926060ee8803fdd64cab7a417
  Stored in directory: /root/.cache/pip/wheels/c1/61/df/768b03527bf006b546dce284eb4249b185669e65afc5fbb2ac
Successfully built fire


In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd
import os, sys

import fire

In [2]:
#%%writefile ../pyscrap_url.py

def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content  #.encode(BeautifulSoup.original_encoding)
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)
    
def get_elements(url, tag='',search={}, fname=None):
    """
    Downloads a page specified by the url parameter
    and returns a list of strings, one per tag element
    """
    
    if isinstance(url,str):
        response = simple_get(url)
    else:
        #if already it is a loaded html page
        response = url

    if response is not None:
        html = BeautifulSoup(response, 'html.parser')
        
        res = []
        if tag:    
            for li in html.select(tag):
                for name in li.text.split('\n'):
                    if len(name) > 0:
                        res.append(name.strip())
                       
                
        if search:
            soup = html            
            
            
            r = ''
            if 'find' in search.keys():
                print('findaing',search['find'])
                soup = soup.find(**search['find'])
                r = soup

                
            if 'find_all' in search.keys():
                print('findaing all of',search['find_all'])
                r = soup.find_all(**search['find_all'])
   
            if r:
                for x in list(r):
                    if len(x) > 0:
                        res.extend(x)
            
        return res

    # Raise an exception if we failed to get any data from the url
    raise Exception('Error retrieving contents at {}'.format(url))    
    
    
if get_ipython().__class__.__name__ == '__main__':
    fire(get_tag_elements)

In [3]:
res = get_elements('https://africafreak.com/100-most-influential-twitter-users-in-africa', tag='h2')

In [4]:
for i in res[100:]:
    res.remove(i)
    #print(res)
names_infl = []
handle_infl = []
for r in res:
    split_data = r.split('.',maxsplit=1)[1].rsplit('(',maxsplit=1)
    name = split_data[0].split(',')[0].strip()
    handle =  split_data[1].split(')',maxsplit=1)[0]
    names_infl.append(name)
    handle_infl.append(handle)
    



In [5]:
df_influencer_handle = pd.DataFrame(handle_infl, columns=["100 influencers handles"])
#df_influencer_handle.to_csv('C:/Users/HP/Desktop/CV, P.Statement and others/10 Academy/100_handles.csv', index=False, header=True)
#print(df_influencer_handle)

In [6]:
url= 'https://www.atlanticcouncil.org/blogs/africasource/african-leaders-respond-to-coronavirus-on-twitter/#east-africa'
response = simple_get(url)

In [7]:
url= 'https://www.atlanticcouncil.org/blogs/africasource/african-leaders-respond-to-coronavirus-on-twitter/#east-africa'
response = get(url).content
re_gov = get_elements(response, tag='blockquote')
names = []
handles = []
for r in re_gov:
    split_data = r.split('— ',maxsplit=1)[1].rsplit('(',maxsplit=1)
    name = split_data[0].split(',')[0].strip()
    handle =  split_data[1].rsplit(')',maxsplit=1)[0]
    names.append(name)
    handles.append(handle)

nam_handle = f'{name}:{handle}'

#df_gov_handle = pd.DataFrame(handles, columns=["Gov influencers handles"])
#df_gov_handle.to_csv('C:/Users/HP/Desktop/CV, P.Statement and others/10 Academy/gov_handles.csv', index=False, header=True)
#print(df_gov_handle)

In [8]:
fl_handles = handles + handle_infl
#final_handle = pd.DataFrame(fl_handles, columns=["combined_handles"])
#final_handle.to_csv (r'C:\Users\HP\Desktop\CV, P.Statement and others\10 Academy\final_handle.csv', index = False, header=True)
#print(final_handle)


In [9]:
import sys
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import re
import string

import matplotlib.dates as mdates
import seaborn as sns
#sns.set()


# to view all columns
#pd.set_option("display.max.columns", None)

In [10]:
#!pip install tweepy
import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

In [11]:
consumer_key = "re8a7dLArl8y2Tic312dbRWxH"
consumer_secret = "DcARRXgLiCfFE0y1aBFF9WlfHtLzZw2TgOCEOkM31rdKG8rJD0"
access_token = "994261917253472256-bRMXtnLl19sVm2a3flZv6t2WaW9b1AG"
access_token_secret = "HMnwkCLauS0OLOVNi62Ay1K9OcVlEvgawcfqOSuIIHbJz"

In [12]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [38]:
#getting tweets
tweets = []
tweetCount=5
for i in handle_Infl:
    try:
        results=api.user_timeline(id=i, count=tweetCount)
    except tweepy.TweepError as e:
                continue  
    for tweet in results:
        tweets.append(tweet.text)
#print(tweets)




In [73]:
# getting followers for 100 influencers
# Calling the get_user function with our parameters
followers = []
for i in handles:
    try:
        results = api.get_user(id=i)
    except tweepy.TweepError as e:
        continue 
    followers.append(results.followers_count)
    #print(results.followers_count)
print(followers)
# followers dataframe
total_followers = pd.DataFrame(followers,columns=["Number of followers"])
total_followers.to_csv('C:/Users/HP/Desktop/CV, P.Statement and others/10 Academy/gov_follower.csv')
print(total_followers.head())




[11324, 39316, 192482, 126, 1599467, 7171, 546580, 2934, 66256, 1104131, 1984380, 424457, 2560, 371105, 4024, 1813811, 3272, 107412, 18642, 340987, 173617, 30169, 256363, 3283, 832993, 10774, 1508566, 737, 8482, 228970, 31806, 116313, 3272430, 1375833, 14915, 1132]
   Number of followers
0                11324
1                39316
2               192482
3                  126
4              1599467


In [80]:
followers = []
for i in handle_infl:
    try:
        results = api.get_user(id=i)
    except tweepy.TweepError as e:
        continue 
    followers.append(results.followers_count)
    #print(results.followers_count)
print(followers)
# followers dataframe
totalinf_followers = pd.DataFrame(followers,columns=["Number of followers"])
totalinf_followers.to_csv('C:/Users/HP/Desktop/CV, P.Statement and others/10 Academy/infl_follower.csv')
print(totalinf_followers.head())

[25705, 31286, 101288, 68073, 21964, 31294, 22798, 8, 50990, 28466, 439432, 52894, 46778, 11927, 24867, 49699, 28444, 117, 17444, 93963, 18239, 21365, 62309, 18499, 19996, 7489, 6, 215386, 232825, 98723, 106165, 70748, 17119, 249964, 48966, 105241, 58232, 80354, 487, 81639, 217261, 192561, 24702, 20448, 191410, 26436, 165773, 1002778, 48253, 50095, 18638, 240621, 105655, 88508, 84533, 1078458, 56242, 127368, 143198, 285565, 131, 215979, 220912, 59207, 29952, 51839, 151981, 114521, 673453, 56876, 541579, 49249, 69, 35228, 938157, 572317, 105683, 183461, 7, 1755568, 1042485, 1160254, 1416120, 200279, 1085295, 1164410, 1060342, 1443391, 3129885, 3578227, 18, 1974635, 10808859]
   Number of followers
0                25705
1                31286
2               101288
3                68073
4                21964


In [76]:
# getting no of likes for gov influencers
likes = []
for i in handles:
    try:
        results = api.get_user(id=i)
    except tweepy.TweepError as e:
        continue
    likes.append(results.favourites_count)
    #print(results.followers_count)
#print(likes)
total_like = pd.DataFrame(likes,columns=["Number of likes"])
total_like.to_csv('C:/Users/HP/Desktop/CV, P.Statement and others/10 Academy/gov_likes.csv')
print(total_like.head())



   Number of likes
0               61
1              915
2              268
3               12
4               63


In [77]:
# getting no of likes for gov influencers
likes = []
for i in handle_infl:
    try:
        results = api.get_user(id=i)
    except tweepy.TweepError as e:
        continue
    likes.append(results.favourites_count)
    #print(results.followers_count)
#print(likes)
total_inflike = pd.DataFrame(likes,columns=["Number of likes"])
total_inflike.to_csv('C:/Users/HP/Desktop/CV, P.Statement and others/10 Academy/infl_likes.csv')
print(total_inflike.head())



   Number of likes
0               10
1             1621
2             9027
3             3916
4             3625


In [15]:
# getting no of following for 100 influencers
following = []
for i in handles:
    try:
        results = api.get_user(id=i)
    except tweepy.TweepError as e:
        continue
    following.append(results.friends_count)

    
#print(following)
gov_following = pd.DataFrame(following,columns=["Number of following"])
gov_following.to_csv('C:/Users/HP/Desktop/CV, P.Statement and others/10 Academy/gov_followings.csv')
print(gov_following.head())



   Number of following
0                   82
1                   26
2                   55
3                  224
4                   14


In [14]:
# getting no of following for 100 influencers
following = []
for i in handle_infl:
    try:
        results = api.get_user(id=i)
    except tweepy.TweepError as e:
        continue
    following.append(results.friends_count)

    
#print(following)
inf_following = pd.DataFrame(following,columns=["Number of following"])
inf_following.to_csv('C:/Users/HP/Desktop/CV, P.Statement and others/10 Academy/inf_followings.csv')
print(inf_following.head())



   Number of following
0                   37
1                 3059
2                 2838
3                 4592
4                 5050


In [16]:
#getting retweets
no_of_retweets = []
for id in fl_handles:
    try:
        tweets = tweepy.Cursor(api.user_timeline, id=i).items()
        for tweet in tweets:
            no_of_retweets.append(tweet.retweet_count)
    except tweepy.TweepError as e:
        continue
#print(no_of_retweets)

#retweets dataframe
gov_retweets = pd.DataFrame(no_of_retweets, columns=["No of retweets"])
gov_retweets.to_csv (r'C:\Users\HP\Desktop\CV, P.Statement and others\10 Academy\gov_retweet.csv', index = False, header=True)
print(gov_retweets.head())


   No of retweets
0              10
1              14
2              10
3              16
4               0


In [ ]:
#getting retweets
no_of_retweets = []
for id in handle_infl:
    try:
        tweets = tweepy.Cursor(api.user_timeline, id=i).items()
        for tweet in tweets:
            no_of_retweets.append(tweet.retweet_count)
    except tweepy.TweepError as e:
        continue
#print(no_of_retweets)

#retweets dataframe
inf_retweets = pd.DataFrame(no_of_retweets, columns=["No of retweets"])
inf_retweets.to_csv (r'C:\Users\HP\Desktop\CV, P.Statement and others\10 Academy\gov_retweet.csv', index = False, header=True)
print(inf_retweets.head())


In [13]:
#getting no of tweets shared
no_tweets_shared = []
for i in handles:
    try:
        results = api.get_user(id=i)
    except tweepy.TweepError as e:
        continue
    no_tweets_shared.append(results.statuses_count)
    
print(no_tweets_shared)

#tweet shared dataframe
gov_statuses = pd.DataFrame(no_tweets_shared, columns=["No of statuses"])
gov_statuses.to_csv (r'C:\Users\HP\Desktop\CV, P.Statement and others\10 Academy\gov_statuses.csv', index = False, header=True)
print(gov_statuses.head())

[1674, 4030, 1087, 125, 18881, 838, 628, 1064, 4718, 9050, 2862, 599, 209, 652, 836, 6644, 732, 40682, 753, 1650, 1738, 5648, 4496, 715, 7139, 1440, 7097, 21, 828, 11168, 30, 349, 4734, 2796, 83, 44, 3771, 16869, 142230, 27324, 19439, 6864, 1638, 1, 21549, 3892, 81200, 16889, 23988, 2166, 2523, 2814, 35262, 23, 4993, 13508, 88730, 42967, 26785, 15627, 9575, 15410, 0, 22994, 7579, 17748, 101293, 4986, 12766, 15755, 8771, 45452, 7187, 35782, 35, 18369, 33459, 14514, 8767, 3906, 48599, 32731, 15884, 265258, 16251, 11100, 8549, 91472, 30378, 23204, 137304, 35966, 10665, 8895, 38395, 53774, 117, 9294, 23704, 13353, 3118, 14758, 3243, 11275, 27250, 14684, 9266, 6476, 1, 21728, 50980, 23582, 27791, 15681, 6, 4525, 19994, 88552, 56044, 45129, 5688, 62840, 144301, 72335, 37213, 322868, 19, 31626, 11186]
     No of statuses
0              1674
1              4030
2              1087
3               125
4             18881
5               838
6               628
7              1064
8             

In [ ]:
#getting no of tweets shared
no_tweets_shared = []
for i in handle_infl:
    try:
        results = api.get_user(id=i)
    except tweepy.TweepError as e:
        continue
    no_tweets_shared.append(results.statuses_count)
    
#print(no_tweets_shared)

#tweet shared dataframe
infl_statuses = pd.DataFrame(no_tweets_shared, columns=["No of statuses"])
infl_statuses.to_csv (r'C:\Users\HP\Desktop\CV, P.Statement and others\10 Academy\infl_statuses.csv', index = False, header=True)
print(infl_statuses.head())

In [13]:
#mention for gov influencers
count = []
for x in range(0, len(handles)):
    name = handles[x]
    mentions_count = []
    try:
        for status in tweepy.Cursor(api.user_timeline, id=name).items():
            entities = status.entities
            if "user_mentions" in entities:
                for ent in entities["user_mentions"]:
                    if ent is not None:
                        if "screen_name" in ent:
                            name = ent["screen_name"]
                            if name is not None:
                                mentions_count.append(name)
    except tweepy.TweepError as e:
        continue
    count.append(len(mentions_count))
    
    
infl_mention = pd.DataFrame(mentions_count, columns=["gov mentions"])
infl_mention.to_csv (r'C:\Users\HP\Desktop\CV, P.Statement and others\10 Academy\infl_statuses.csv', index = False, header=True)
print(infl_mention.head())

Empty DataFrame
Columns: [gov mentions]
Index: []


In [ ]:
#mention for top 100 influncers
count = []
for x in range(0, len(handle_infl)):
    name = handle_infl[x]
    mentions_count = []
    try:
       for status in tweepy.Cursor(api.user_timeline, id=name).items():
         entities = status.entities
         if "user_mentions" in entities:
            for ent in entities["user_mentions"]:
              if ent is not None:
                if "screen_name" in ent:
                  name = ent["screen_name"]
                  if name is not None:
                    mentions_count.append(name)
    except tweepy.TweepError as e:
        continue
    count.append(len(mentions_count))

infl_statuses = pd.DataFrame(no_tweets_shared, columns=["No of statuses"])
infl_statuses.to_csv (r'C:\Users\HP\Desktop\CV, P.Statement and others\10 Academy\infl_statuses.csv', index = False, header=True)
print(infl_statuses.head())

In [ ]:
def get_hashtags(fl_handles):
    cols = ['id', 'name', 'screen_name', 'hashtags']
    # dataframe that would be returned at the end
    df = pd.DataFrame(columns=cols)
    handle_data = []
    if len(fl_handle) > 0: 
        for handle in fl_handles:
            value_list = {}
            print("Getting hashtags for " + handle)
            # this helps avoid Tweepy errors like suspended users or user not found errors
            try:
                item = api.get_user(handle)
            except tweepy.TweepError as e:
                continue
            value_list['id'] = item.id_str
            value_list['name'] = item.name
            value_list['screen_name'] = item.screen_name
            #get average daily tweets
            no_tweets = item.statuses_count
            account_created_date = item.created_at
            delta = datetime.utcnow() - account_created_date
            account_age_days = delta.days
            hashtags = set()
            hash_dic = {}
            tweet_count = 0
            end_date = datetime.utcnow() - timedelta(days=30)
            for status in tweepy.Cursor(api.user_timeline, id=twitter_handle).items():
                if hasattr(status, "entities"):
                    entities = status.entities
                # get hashtags
                if "hashtags" in entities:
                    for ent in entities["hashtags"]:
                        if ent is not None:
                            if "text" in ent:
                                hashtag = ent["text"]
                                if hashtag is not None:
                                    if hashtag in hashtags:
                                        hash_dic[hashtag]+=1
                                    else:
                                        hashtags.add(hashtag)
                                        hash_dic[hashtag] = 1
                value_list['hashtags'] = hash_dic
            try:
                #escape handles with no hashtags
                df = df.append(pd.DataFrame(value_list))
                # the code snippet below before the return is used to save to file due to constant connections loss
                new_df = df.reset_index().rename(columns={'hashtags':'hashtags_count','index':'hashtags'})
            except tweepy.TweepError as e:
                continue
    return df.reset_index().rename(columns={'hashtags':'hashtags_count','index':'hashtags'})

### Popularity reach
### Reach Score
### Relevance Score

In [22]:
#popularity reach = retweets + likes
popularity = pd.concat([df_retweets,df_likes], axis=1)
#print(popularity)
popularity["Popularity_score"] = popularity["No of retweets"]+popularity["No of likes"]
print(popularity)





       No of retweets  No of likes  Popularity_score
0                 497         61.0             558.0
1                2749        915.0            3664.0
2               93077        268.0           93345.0
3                6013         12.0            6025.0
4                1651         63.0            1714.0
...               ...          ...               ...
14927              73          NaN               NaN
14928               3          NaN               NaN
14929             670          NaN               NaN
14930              70          NaN               NaN
14931             112          NaN               NaN

[14932 rows x 3 columns]


In [22]:
#Reach Score = followers - following
reach = pd.concat([df_fl_handles,df_followers,d_following], axis=1)
print(reach.head())
reach['reach_score']= reach["Number of followers"] - reach["Number of following"]
print(reach.head()) 


NameError: name 'df_followers' is not defined

## Web scrapping using bash script
If the web site has a quite simple HTML, you can easily use curl to perform the request and then extract the needed values using bash commands grep, cut , sed, ..

This tutorial is adapted from [this](https://medium.com/@LiliSousa/web-scraping-with-bash-690e4ee7f98d) medium article

In [ ]:
%%bash 

# curl the page and save content to tmp_file
#url = "https://www.atlanticcouncil.org/blogs/africasource/african-leaders-respond-to-coronavirus-on-twitter/#east-africa"
#curl -X GET $url -o tmp_file


#!/bin/bash

# write headers to CSV file
echo "Name, twitter_id" >> extractData.csv
n="1"
while [ $n -lt 2 ]
do
  
  #get title
  title=$(cat tmp_file | grep "class=\"twitter-tweet\"" | cut -d ';' -f1 )
  echo $title
  #get author
  #twitter_id=$(cat tmp_file |grep -A1 "class=\"css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0\"" | tail -1)

  #echo "$title, $twitter_id" >> extractData.csv
  #echo "$title, $twitter_id"
    
  n=$[$n+1]

done